In [113]:
import numpy as np
import plotly.graph_objects as go


# Функции

In [114]:
def sigmoid(x):
    return 1/ (1 + np.exp(-x))

def savePoints(points, filename):
    with open(filename, 'w') as f:
        for point in points:
            f.write(f"{point[0]} {point[1]} {point[2]}\n")

def loadPoints(filename):
    points = []
    with open(filename, 'r') as f:
        for line in f:
            x, y, z = map(float, line.split())
            points.append((x, y, z))
    return points

def Relu(x):
    return np.maximum(0, x)

# Генерация точек

In [115]:
def generate_points(n):
    points = []
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        z = np.random.uniform(-1, 1)
        points.append((x, y, z))
    return points

# Генерация точек

In [116]:

# savePoints(generate_points(1000), 'points_training.txt')

# Визуализация точек

In [117]:
def interactive_3d_plot(points):
    octant_data = [[] for _ in range(8)]
    
    for x, y, z in points:
        octant = 0
        if x < 0: octant += 1
        if y < 0: octant += 2  
        if z < 0: octant += 4
        octant_data[octant].append((x, y, z))
    
    fig = go.Figure()

    fig.add_trace(go.Scatter3d(x=[-1, 1], y=[0, 0], z=[0, 0], 
                              mode='lines', line=dict(color='black', width=5), 
                              name='Ось X'))
    fig.add_trace(go.Scatter3d(x=[0, 0], y=[-1, 1], z=[0, 0], 
                              mode='lines', line=dict(color='black', width=5), 
                              name='Ось Y'))
    fig.add_trace(go.Scatter3d(x=[0, 0], y=[0, 0], z=[-1, 1], 
                              mode='lines', line=dict(color='black', width=5), 
                              name='Ось Z'))
    
    colors = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'pink', 'gray']
    names = ['(+,+,+)', '(-,+,+)', '(+,-,+)', '(-,-,+)', 
             '(+,+,-)', '(-,+,-)', '(+,-,-)', '(-,-,-)']
    
    for i, (data, color, name) in enumerate(zip(octant_data, colors, names)):
        if data:
            x_coords, y_coords, z_coords = zip(*data)
            fig.add_trace(go.Scatter3d(
                x=x_coords, y=y_coords, z=z_coords,
                mode='markers',
                marker=dict(size=8, color=color),
                name=f'Октант {i+1} {name}'
            ))
    
    fig.update_layout(
        width=800, height=800,
        title='Интерактивная 3D визуализация',
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z')
    )
    
    fig.show()

In [118]:
interactive_3d_plot(loadPoints('points_training.txt'))

In [119]:
interactive_3d_plot(loadPoints('points_inference.txt'))

# Открытие файлов с точками и присвоение классов

In [120]:
def getOctant(x,y,z):
    octant = 0
    if x < 0: octant += 1
    if y < 0: octant += 2
    if z < 0: octant += 4
    return octant + 1

def loadPointsWithClasses(filename):
    points = []
    octants = []
    with open(filename, 'r') as f:
        for line in f:
            x, y, z = map(float, line.split())
            points.append((x, y, z))
            octants.append(getOctant(x, y, z))
    return np.array(points), np.array(octants) 

def makeTargetVector(octants):
    targets = np.zeros((len(octants), 8))
    for i, octant in enumerate(octants):
        targets[i][octant-1] = 1
    return targets

xTrain, yClasses = loadPointsWithClasses('points_training.txt')
yTrain = makeTargetVector(yClasses) 

print(f"Точка: {xTrain[0]}")
print(f"Октант: {yClasses[0]}")  
print(f"Целевой вектор: {yTrain[0]}")

Точка: [ 0.89286915 -0.22905191 -0.11924598]
Октант: 7
Целевой вектор: [0. 0. 0. 0. 0. 0. 1. 0.]


# Перцептрон

In [121]:
class Perceptron:
    def __init__(self, input_size):
        self.weights = np.random.rand(8, input_size)
        self.bias = np.random.rand(8)

    def forward(self, x):
        return sigmoid(np.dot(self.weights, x) + self.bias)

    def train(self, x, y, y_true, learning_rate=0.01, target_accuracy=0.95, weight_decay=0.001, max_epochs=400):
        epoch = 0
        while epoch < max_epochs:
            for i in range(len(x)):
                y_pred = self.forward(x[i])
                for j in range(8):
                    error = y[i][j] - y_pred[j]
                    self.weights[j] = self.weights[j] + learning_rate * error * np.array(x[i]) * (1-weight_decay)
                    self.bias[j] += learning_rate * error
            
            if epoch % 10 == 0:
                correct = 0
                for i in range(len(x)):
                    prediction = np.argmax(self.forward(x[i])) + 1
                    if prediction == y_true[i]:
                        correct += 1
                accuracy = correct / len(x)
                print(f"Эпоха {epoch}, точность: {accuracy:.2%}")
                
                if accuracy >= target_accuracy:
                    print(f"Достигнута целевая точность {target_accuracy:.0%}")
                    break
            
            epoch += 1


    def saveWeights(self, filename):
        np.savetxt(filename + '.txt', self.weights)
        np.savetxt(filename + '_bias.txt', self.bias)

    def loadWeights(self, filename):
        self.weights = np.loadtxt(filename + '.txt')
        self.bias = np.loadtxt(filename + '_bias.txt')
    
    def printResults(self, x, y):
        correct = 0
        for i in range(len(x)):
            output = self.forward(x[i])
            prediction = np.argmax(output) + 1
            print(f"Точка: {x[i]}")
            print(f"Реальный октант: {y[i]}")
            print(f"Предсказание: {prediction}")
            if prediction == y[i]:
                correct += 1
            print()
        
        print(f"Точность: {correct}/{len(y)} = {correct/len(y):.2%}")

    def getAnswerForOnePoint(self, x, y, z):
        point = np.array([x, y, z])
        output = self.forward(point)
        print(f"Ответ: {np.argmax(output) + 1}")


# Реализация

In [122]:
xInference, yInference = loadPointsWithClasses('points_inference.txt')

perceptron = Perceptron(3)
perceptron.train(xTrain, yTrain, yClasses)
perceptron.saveWeights('weights')
perceptron.printResults(xInference, yInference)

Эпоха 0, точность: 71.50%
Эпоха 10, точность: 94.10%
Эпоха 20, точность: 94.90%
Эпоха 30, точность: 95.30%
Достигнута целевая точность 95%
Точка: [0.77713774 0.1630673  0.02121406]
Реальный октант: 1
Предсказание: 1

Точка: [0.89820353 0.85970497 0.67630812]
Реальный октант: 1
Предсказание: 1

Точка: [ 0.95817417  0.07873563 -0.05771293]
Реальный октант: 5
Предсказание: 5

Точка: [0.33069516 0.20559343 0.59636621]
Реальный октант: 1
Предсказание: 1

Точка: [-0.32548394  0.81423516 -0.76531842]
Реальный октант: 6
Предсказание: 6

Точка: [0.37742891 0.57619167 0.82238806]
Реальный октант: 1
Предсказание: 1

Точка: [-0.72216871  0.55858439 -0.12417909]
Реальный октант: 6
Предсказание: 6

Точка: [-0.24043499  0.2679133   0.22339181]
Реальный октант: 2
Предсказание: 2

Точка: [ 0.20655855  0.34916856 -0.66664467]
Реальный октант: 5
Предсказание: 5

Точка: [-0.93097335  0.6592487   0.2650316 ]
Реальный октант: 2
Предсказание: 2

Точка: [ 0.70428692  0.04287356 -0.22516154]
Реальный октант: 5

In [128]:
perceptron2 = Perceptron(3)
perceptron2.loadWeights('weights')
perceptron2.getAnswerForOnePoint(-0.11, 0.3, -1)

Ответ: 5
